In [2]:
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sn
%matplotlib inline
import numpy as np
#from wordcloud import WordCloud
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from IPython.display import display, HTML
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
import itertools
#import gensim
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold

import nltk
from nltk.corpus import stopwords
import re


import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob

# Processing tweets from news sites in the US

In [4]:
#load all tweets into one df

In [28]:
path = r'D:/NEU/Individual Projects/Covid19 Project/News sites tweets data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

nstweets = pd.concat(li, axis=0, ignore_index=True)

In [29]:
print('No. of tweets and columns in the df are:')
print(nstweets.shape)

No. of tweets and columns in the df are:
(54752, 3)


In [30]:
nstweets.head()

,Datetime,Tweet Id,Text
0,2020-10-21 02:34:06,1318742328757059584,"""Although it is a serious disease, I feel fort..."
1,2020-10-21 02:23:04,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...
2,2020-10-21 02:17:07,1318738053696028672,Breonna Taylor grand jury members say they wer...
3,2020-10-21 02:07:04,1318735525109506048,Security footage shows a disgruntled customer ...
4,2020-10-21 01:51:04,1318731497415806977,"""They really run the risk of becoming extinct ..."


In [31]:
#separate datetime into date and time columns

nstweets[['Date','Time']] = nstweets.Datetime.str.split(expand=True) 

nstweets.head()

,Datetime,Tweet Id,Text,Date,Time
0,2020-10-21 02:34:06,1318742328757059584,"""Although it is a serious disease, I feel fort...",2020-10-21,02:34:06
1,2020-10-21 02:23:04,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...,2020-10-21,02:23:04
2,2020-10-21 02:17:07,1318738053696028672,Breonna Taylor grand jury members say they wer...,2020-10-21,02:17:07
3,2020-10-21 02:07:04,1318735525109506048,Security footage shows a disgruntled customer ...,2020-10-21,02:07:04
4,2020-10-21 01:51:04,1318731497415806977,"""They really run the risk of becoming extinct ...",2020-10-21,01:51:04


In [32]:
#remoce datetime column

del nstweets['Datetime']


In [33]:
tweets = nstweets

In [34]:
# extracting hashtags

tweets['hashtag'] = tweets['Text'].apply(lambda x: re.findall(r"#(\w+)", x))

In [36]:
# removing URLs, mentions, emojis etc
import preprocessor as p

tweets['Text'] = tweets['Text'].apply(lambda x: p.clean(x))

In [40]:
#remove punctuation

import string

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

tweets['Tweet_punct'] = tweets['Text'].apply(lambda x: remove_punct(x))


,Datetime,Tweet Id,Text
0,2020-10-21 01:39:50,1318728672216571905,Judge rejects settlement over stalled student ...
1,2020-10-21 01:02:51,1318719365076951043,Opinion: Progressives should support Biden now...
2,2020-10-21 00:31:06,1318711371454963718,Breonna Taylor grand jury was not given option...
3,2020-10-21 00:22:02,1318709090516946944,How parents can talk to college kids about cov...
4,2020-10-20 23:46:05,1318700045261295616,"Analysis: We are not ""rounding the corner"" on ..."


In [42]:
#tokenize the tweets

def tokenization(text):
    text = re.split('\W+', text)
    return text

tweets['Tweet_tokenized'] = tweets['Tweet_punct'].apply(lambda x: tokenization(x.lower()))
tweets.head()


,Tweet Id,Text,Date,Time,hashtag,Tweet_punct,Tweet_tokenized
0,1318742328757059584,"""Although it is a serious disease, I feel fort...",2020-10-21,02:34:06,[],Although it is a serious disease I feel fortun...,"[although, it, is, a, serious, disease, i, fee..."
1,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...,2020-10-21,02:23:04,[],AUTUMN COLORS Aerial footage captured by a dro...,"[autumn, colors, aerial, footage, captured, by..."
2,1318738053696028672,Breonna Taylor grand jury members say they wer...,2020-10-21,02:17:07,[],Breonna Taylor grand jury members say they wer...,"[breonna, taylor, grand, jury, members, say, t..."
3,1318735525109506048,Security footage shows a disgruntled customer ...,2020-10-21,02:07:04,[],Security footage shows a disgruntled customer ...,"[security, footage, shows, a, disgruntled, cus..."
4,1318731497415806977,"""They really run the risk of becoming extinct ...",2020-10-21,01:51:04,[],They really run the risk of becoming extinct i...,"[they, really, run, the, risk, of, becoming, e..."


In [44]:
# remove stop words

stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
tweets['Tweet_nostop'] = tweets['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweets.head()

,Tweet Id,Text,Date,Time,hashtag,Tweet_punct,Tweet_tokenized,Tweet_nostop
0,1318742328757059584,"""Although it is a serious disease, I feel fort...",2020-10-21,02:34:06,[],Although it is a serious disease I feel fortun...,"[although, it, is, a, serious, disease, i, fee...","[although, serious, disease, feel, fortunate, ..."
1,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...,2020-10-21,02:23:04,[],AUTUMN COLORS Aerial footage captured by a dro...,"[autumn, colors, aerial, footage, captured, by...","[autumn, colors, aerial, footage, captured, dr..."
2,1318738053696028672,Breonna Taylor grand jury members say they wer...,2020-10-21,02:17:07,[],Breonna Taylor grand jury members say they wer...,"[breonna, taylor, grand, jury, members, say, t...","[breonna, taylor, grand, jury, members, say, p..."
3,1318735525109506048,Security footage shows a disgruntled customer ...,2020-10-21,02:07:04,[],Security footage shows a disgruntled customer ...,"[security, footage, shows, a, disgruntled, cus...","[security, footage, shows, disgruntled, custom..."
4,1318731497415806977,"""They really run the risk of becoming extinct ...",2020-10-21,01:51:04,[],They really run the risk of becoming extinct i...,"[they, really, run, the, risk, of, becoming, e...","[really, run, risk, becoming, extinct, inside,..."


In [45]:
# stemming and lemmatization

ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tweets['Tweet_stemmed'] = tweets['Tweet_nostop'].apply(lambda x: stemming(x))
tweets.head()

,Tweet Id,Text,Date,Time,hashtag,Tweet_punct,Tweet_tokenized,Tweet_nostop,Tweet_stemmed
0,1318742328757059584,"""Although it is a serious disease, I feel fort...",2020-10-21,02:34:06,[],Although it is a serious disease I feel fortun...,"[although, it, is, a, serious, disease, i, fee...","[although, serious, disease, feel, fortunate, ...","[although, seriou, diseas, feel, fortun, great..."
1,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...,2020-10-21,02:23:04,[],AUTUMN COLORS Aerial footage captured by a dro...,"[autumn, colors, aerial, footage, captured, by...","[autumn, colors, aerial, footage, captured, dr...","[autumn, color, aerial, footag, captur, drone,..."
2,1318738053696028672,Breonna Taylor grand jury members say they wer...,2020-10-21,02:17:07,[],Breonna Taylor grand jury members say they wer...,"[breonna, taylor, grand, jury, members, say, t...","[breonna, taylor, grand, jury, members, say, p...","[breonna, taylor, grand, juri, member, say, pr..."
3,1318735525109506048,Security footage shows a disgruntled customer ...,2020-10-21,02:07:04,[],Security footage shows a disgruntled customer ...,"[security, footage, shows, a, disgruntled, cus...","[security, footage, shows, disgruntled, custom...","[secur, footag, show, disgruntl, custom, destr..."
4,1318731497415806977,"""They really run the risk of becoming extinct ...",2020-10-21,01:51:04,[],They really run the risk of becoming extinct i...,"[they, really, run, the, risk, of, becoming, e...","[really, run, risk, becoming, extinct, inside,...","[realli, run, risk, becom, extinct, insid, lif..."


In [46]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

tweets['Tweet_lemmatized'] = tweets['Tweet_nostop'].apply(lambda x: lemmatizer(x))
tweets.head()

,Tweet Id,Text,Date,Time,hashtag,Tweet_punct,Tweet_tokenized,Tweet_nostop,Tweet_stemmed,Tweet_lemmatized
0,1318742328757059584,"""Although it is a serious disease, I feel fort...",2020-10-21,02:34:06,[],Although it is a serious disease I feel fortun...,"[although, it, is, a, serious, disease, i, fee...","[although, serious, disease, feel, fortunate, ...","[although, seriou, diseas, feel, fortun, great...","[although, serious, disease, feel, fortunate, ..."
1,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...,2020-10-21,02:23:04,[],AUTUMN COLORS Aerial footage captured by a dro...,"[autumn, colors, aerial, footage, captured, by...","[autumn, colors, aerial, footage, captured, dr...","[autumn, color, aerial, footag, captur, drone,...","[autumn, color, aerial, footage, captured, dro..."
2,1318738053696028672,Breonna Taylor grand jury members say they wer...,2020-10-21,02:17:07,[],Breonna Taylor grand jury members say they wer...,"[breonna, taylor, grand, jury, members, say, t...","[breonna, taylor, grand, jury, members, say, p...","[breonna, taylor, grand, juri, member, say, pr...","[breonna, taylor, grand, jury, member, say, pr..."
3,1318735525109506048,Security footage shows a disgruntled customer ...,2020-10-21,02:07:04,[],Security footage shows a disgruntled customer ...,"[security, footage, shows, a, disgruntled, cus...","[security, footage, shows, disgruntled, custom...","[secur, footag, show, disgruntl, custom, destr...","[security, footage, show, disgruntled, custome..."
4,1318731497415806977,"""They really run the risk of becoming extinct ...",2020-10-21,01:51:04,[],They really run the risk of becoming extinct i...,"[they, really, run, the, risk, of, becoming, e...","[really, run, risk, becoming, extinct, inside,...","[realli, run, risk, becom, extinct, insid, lif...","[really, run, risk, becoming, extinct, inside,..."


In [47]:
#Save the tweets dataframe

tweets.to_csv(r'D:/NEU/Individual Projects/Covid19 Project/cleanednstweets.csv', index = False)

In [48]:
tweets.shape

(54752, 10)

In [53]:
# Have to label news sites for tweets manually

news_site = [None]*tweets.shape[0]

for i in range(0,3234):
    news_site[i] = 'ABC'
    
for i in range(3234,6466):
    news_site[i] = 'BBC'

for i in range(6466,9695):
    news_site[i] = 'Breitbart'
    
for i in range(9695,12929):
    news_site[i] = 'CBS'
    
for i in range(12929,16162):
    news_site[i] = 'CNN'
    
for i in range(16162,19398):
    news_site[i] = 'FiveThirtyEight'

for i in range(19398,22632):
    news_site[i] = 'Fox'
    
for i in range(22632,25879):
    news_site[i] = 'NBC'
    
for i in range(25879,29119):
    news_site[i] = 'NPR'
    
for i in range(29119,32324):
    news_site[i] = 'Nytimes'
    
for i in range(32324,35527):
    news_site[i] = 'Politico'
    
for i in range(35527,38732):
    news_site[i] = 'Reuters'
    
for i in range(38732,41960):
    news_site[i] = 'The Atlantic'
    
for i in range(41960,45184):
    news_site[i] = 'The Onion'
    
for i in range(45184,48401):
    news_site[i] = 'Time'
    
for i in range(48401,51575):
    news_site[i] = 'VICE'
    
for i in range(51575,54752):
    news_site[i] = 'Washington Post'

In [54]:
tweets['news_site'] = news_site
tweets.head()

,Tweet Id,Text,Date,Time,hashtag,Tweet_punct,Tweet_tokenized,Tweet_nostop,Tweet_stemmed,Tweet_lemmatized,news_site
0,1318742328757059584,"""Although it is a serious disease, I feel fort...",2020-10-21,02:34:06,[],Although it is a serious disease I feel fortun...,"[although, it, is, a, serious, disease, i, fee...","[although, serious, disease, feel, fortunate, ...","[although, seriou, diseas, feel, fortun, great...","[although, serious, disease, feel, fortunate, ...",ABC
1,1318739549116375040,AUTUMN COLORS: Aerial footage captured by a dr...,2020-10-21,02:23:04,[],AUTUMN COLORS Aerial footage captured by a dro...,"[autumn, colors, aerial, footage, captured, by...","[autumn, colors, aerial, footage, captured, dr...","[autumn, color, aerial, footag, captur, drone,...","[autumn, color, aerial, footage, captured, dro...",ABC
2,1318738053696028672,Breonna Taylor grand jury members say they wer...,2020-10-21,02:17:07,[],Breonna Taylor grand jury members say they wer...,"[breonna, taylor, grand, jury, members, say, t...","[breonna, taylor, grand, jury, members, say, p...","[breonna, taylor, grand, juri, member, say, pr...","[breonna, taylor, grand, jury, member, say, pr...",ABC
3,1318735525109506048,Security footage shows a disgruntled customer ...,2020-10-21,02:07:04,[],Security footage shows a disgruntled customer ...,"[security, footage, shows, a, disgruntled, cus...","[security, footage, shows, disgruntled, custom...","[secur, footag, show, disgruntl, custom, destr...","[security, footage, show, disgruntled, custome...",ABC
4,1318731497415806977,"""They really run the risk of becoming extinct ...",2020-10-21,01:51:04,[],They really run the risk of becoming extinct i...,"[they, really, run, the, risk, of, becoming, e...","[really, run, risk, becoming, extinct, inside,...","[realli, run, risk, becom, extinct, insid, lif...","[really, run, risk, becoming, extinct, inside,...",ABC


In [56]:
# remove tweets from the onion

indices = tweets[tweets['news_site'] == 'The Onion'].index
tweets.drop(indices, inplace = True)

#Save the tweets dataframe

tweets.to_csv(r'D:/NEU/Individual Projects/Covid19 Project/latest_clean_tweets.csv', index = False)